<a href="https://colab.research.google.com/github/jitendra-manwani/GenAI-architect/blob/main/Jitendra_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Assignment 3
# Meeting Transcript Summarizer & Action-Item Extractor


# Step 1: Install required packages
!pip install openai pydantic

In [10]:
# Step 2: Import necessary libraries
from openai import AzureOpenAI
import json
from pydantic import BaseModel
from typing import List

In [11]:
# Step 3: Set up Azure OpenAI configuration
# Replace these with your actual Azure OpenAI credentials
AZURE_OPENAI_ENDPOINT = "https://eastus.api.cognitive.microsoft.com/"
AZURE_OPENAI_KEY = "63a0587ef7b24ca9b5d7b179c13fbaab"
AZURE_DEPLOYMENT_NAME = "telcogpt"

# Create Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY,
    api_version="2024-12-01-preview"
)

In [5]:
# Step 4: Define Pydantic model for validation
class MeetingNotes(BaseModel):
    summary: str
    action_items: List[str]

In [6]:
# Step 5: Sample test transcripts (from the assignment)
sample_transcript_1 = """Host: Let's kick off our marketing sync.
Emma: The social campaign draft is 80% done; I'll share it today.
Frank: I spoke with the design team—they'll deliver assets by Tuesday.
Emma: Once we have assets, I'll schedule the ads for next week.
George: Reminder: submit your budget requests before end of day.
Host: Noted. I'll send out the final budget spreadsheet."""

sample_transcript_2 = """Alice: Welcome everyone. Today we need to finalize the Q3 roadmap.
Bob: I've emailed the updated feature list—please review by Friday.
Carol: I'll set up the user‐testing sessions next week.
Dan: Let's push the new UI mockups to staging on Wednesday.
Alice: Great. Also, can someone compile the stakeholder feedback into a slide deck?
Bob: I can handle the slide deck by Monday.
Alice: Thanks, team. Meeting adjourned."""

sample_transcript_3 = """Priya: We've identified a bottleneck in the checkout API.
Carlos: I'll benchmark the current response times by tomorrow.
Priya: After that, optimize the database indices.
Carlos: Sure, I'll propose index changes by Thursday.
Sara: Also, draft a rollback plan in case of production issues.
Carlos: I'll include that in my ticket."""

sample_transcript_4 = """HR: We need to update the travel‐expense policy.
Alex: I'll draft the new policy doc and share with legal.
HR: Please include guidelines on per diem limits.
Legal: I'll review and provide feedback within three days.
HR: Once finalized, schedule a companywide announcement."""

sample_transcript_5 = """John: Sales numbers for April are in—up 12% over target.
Lia: I'll distribute the detailed report to leadership this afternoon.
Mike: We need to train two new reps to handle increased volume.
Lia: I'll coordinate recruiting with HR by end of week.
John: Excellent. Let's aim to onboard them by May 15th."""

In [7]:
# Step 6: Create the prompt template
def create_prompt(transcript):
    prompt = f"""You are a meeting assistant.
1. Summarize the meeting transcript below in exactly two sentences.
2. Then list all action items mentioned, each as a separate bullet beginning with a dash.
Return the result strictly as JSON with keys "summary" and "action_items".

Transcript:
{transcript}"""
    return prompt


In [12]:
# Step 7: Main processing function
def process_transcript(transcript):
    try:
        # Create the prompt
        prompt = create_prompt(transcript)

        # Call Azure OpenAI API
        response = client.chat.completions.create(
            model=AZURE_DEPLOYMENT_NAME,
            messages=[
                {"role": "system", "content": "You are a helpful meeting assistant that extracts summaries and action items."},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
            max_tokens=500
        )

        # Extract the response content
        content = response.choices[0].message.content.strip()
        print("Raw AI Response:")
        print(content)
        print("\n" + "="*50 + "\n")

        # Parse JSON
        parsed_json = json.loads(content)

        # Validate with Pydantic
        meeting_notes = MeetingNotes(**parsed_json)

        return meeting_notes.model_dump()

    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        print("Retrying with stricter prompt...")

        # Retry with stricter prompt
        try:
            retry_prompt = f"""Please output valid JSON only. No other text.

{prompt}"""

            response = client.chat.completions.create(
                model=AZURE_DEPLOYMENT_NAME,
                messages=[
                    {"role": "system", "content": "You must respond with valid JSON only. No additional text."},
                    {"role": "user", "content": retry_prompt}
                ],
                temperature=0,
                max_tokens=500
            )

            content = response.choices[0].message.content.strip()
            parsed_json = json.loads(content)
            meeting_notes = MeetingNotes(**parsed_json)
            return meeting_notes.model_dump()

        except Exception as retry_error:
            return {"error": f"Failed to parse after retry: {retry_error}"}

    except Exception as e:
        return {"error": f"API call failed: {e}"}

In [13]:
# Step 8: Test with sample transcripts
print("Testing Meeting Transcript Summarizer")
print("="*50)

# Test Sample 1
print("SAMPLE 1 - Marketing Sync:")
result1 = process_transcript(sample_transcript_1)
print("Result:")
print(json.dumps(result1, indent=2))
print("\n")

# Test Sample 2
print("SAMPLE 2 - Q3 Roadmap:")
result2 = process_transcript(sample_transcript_2)
print("Result:")
print(json.dumps(result2, indent=2))
print("\n")

# Test Sample 3
print("SAMPLE 3 - API Bottleneck:")
result3 = process_transcript(sample_transcript_3)
print("Result:")
print(json.dumps(result3, indent=2))
print("\n")

# Test Sample 4
print("SAMPLE 4 - Travel Policy:")
result4 = process_transcript(sample_transcript_4)
print("Result:")
print(json.dumps(result4, indent=2))
print("\n")

# Test Sample 5
print("SAMPLE 5 - Sales Report:")
result5 = process_transcript(sample_transcript_5)
print("Result:")
print(json.dumps(result5, indent=2))
print("\n")

Testing Meeting Transcript Summarizer
SAMPLE 1 - Marketing Sync:
Raw AI Response:
{
  "summary": "The marketing sync focused on the progress of the social campaign and the timeline for asset delivery. Key reminders included submitting budget requests and the host's commitment to send out the final budget spreadsheet.",
  "action_items": [
    "- Emma will share the social campaign draft today.",
    "- Frank will ensure the design team delivers assets by Tuesday.",
    "- Emma will schedule the ads for next week once the assets are received.",
    "- Team members need to submit their budget requests before the end of the day.",
    "- The host will send out the final budget spreadsheet."
  ]
}


Result:
{
  "summary": "The marketing sync focused on the progress of the social campaign and the timeline for asset delivery. Key reminders included submitting budget requests and the host's commitment to send out the final budget spreadsheet.",
  "action_items": [
    "- Emma will share the s